In [ ]:
%load_ext lab_black

In [ ]:
import ohawf
from apiclient.discovery import build

In [ ]:
creds = ohawf.get()

In [ ]:
service = build('analytics', 'v3', credentials=creds)

In [ ]:
accounts = service.management().accounts().list().execute()

In [ ]:
for adict in accounts["items"]:
    print(f"Name: {adict['name']}")
    print(f"ID: {adict['id']}")
    print()

In [ ]:
# Say you had a file with the Profile IDs you want metrics for:
with open("profiles.txt") as fh:
    profiles = [x.strip() for x in fh.readlines()]

In [ ]:
start_date = "2022-01-01"
end_date = "2022-12-31"

In [ ]:
from collections import namedtuple
from sqlitedict import SqliteDict as sqldict

In [ ]:
# Use the arguments for your API also as dict keys
Args = namedtuple("Args", "profile, npt")

In [ ]:
# Create the keys for uncollected data

with sqldict("ga.db") as db:
    for profile in profiles:
        profile = int(profile)
        for npt in range(1, 50001, 10000):
            arg = Args(profile, npt)
            if str(arg) not in db:
                print(arg)
                db[str(arg)] = None
        db.commit()

In [ ]:
# Function that hits the Google Analytics V3 API
def ga(profile, npt):
    return (
        service.data()
        .ga()
        .get(
            ids=f"ga:{profile}",
            start_date=start_date,
            end_date=end_date,
            dimensions="ga:landingPagePath",
            segment="gaid::-5",
            metrics="ga:users,ga:newusers,ga:sessions,ga:bouncerate,ga:pageviewsPerSession,ga:avgSessionDuration",
            sort="-ga:sessions",
            # filters="ga:medium==organic",
            start_index=f"{npt}",
            max_results="10000",
        )
        .execute()
    )

In [ ]:
# Collect raw data into database
with sqldict("ga.db") as db:
    for key in db:
        tkey = eval(key)
        profile, npt = tkey
        data = db[key]
        if data == None:
            print(f"Hitting API for profile: {profile} npt: {npt}")
            response = ga(profile, npt)
            db[key] = response
            db.commit()
            sleep(10)